In [6]:
from time import time

import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import torch

from model_mnist import MLP, BinarizedMLP
from mnist_tools import train, test
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import os
import time
import pickle
from math import floor
from datetime import datetime


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
epochs = 10
lr = 1.0

In [8]:
optimizers = {
    "Adam": optim.Adam,
    "AdaMax": optim.Adamax,
    "AdaDelta": optim.Adadelta
}

models = {
    "Classic": MLP,
    "Binary": BinarizedMLP
}


In [9]:
train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": batch_size}

if device == "cuda":
    cuda_kwargs = {
        "num_workers": 1,
        "pin_memory": True,
        "shuffle": True
        }

    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


# Dataset configuration
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST('./mnist', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./mnist', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, **train_kwargs)
test_loader = DataLoader(test_dataset, **test_kwargs)


In [10]:
from recorder import setup_logging
from time import time


results = []

for model_name, model_class in models.items():
    for optimizer_name, optimizer in optimizers.items():
        
        # Set up logging
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        log_filename = f"{model_name}_{optimizer_name}_{timestamp}.txt"
        logger = setup_logging(log_filename)
               
        model_instance = model_class().to(device)
        optimizer = optimizer(model_instance.parameters(), lr=lr)
        
        scheduler = StepLR(optimizer, step_size=1, gamma=0.7) # default gamma by authors

        for epoch in range(1, epochs+1):

            start_train_time = time()
            
            train_loss, train_accuracy = train(False, model_instance, device, train_loader, optimizer, epoch, 100, logger)
            train_time = time() - start_train_time

            start_test_time = time()
            test_loss, test_accuracy = test(model_instance, device, test_loader, logger)
            test_time = time() - start_train_time
            
            scheduler.step()

            results.append({
                "model_name": model_name,
                "optimizer_name": optimizer_name,
                "epoch": epoch,
                "train_loss": train_loss,
                "train_acc": train_accuracy,
                "test_loss": test_loss,
                "test_acc": test_accuracy,
                "epoch_train_time": train_time,
                "epoch_test_time": test_time
            })

df_results = pd.DataFrame(results)
df_results.to_csv("mnist_experiment_results.csv", index=False)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.333808
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.821298
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.491258
Train Epoch: 1 [19200/60000 (32%)]	Loss: 3.589172
Train Epoch: 1 [25600/60000 (43%)]	Loss: 11.720989
Train Epoch: 1 [32000/60000 (53%)]	Loss: 3.437359
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.311002
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.471744
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.544994
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.246906

Test set: Average loss: 10302.5102, Accuracy: 7587/10000 (76%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.719995
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.442071
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.306730
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.634135
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.274535
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.513435
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.413619
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.808810
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.427